# Assignment 3

## Get Preprocessed Data

In [5]:
import preprocess
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

train_target, train_data, test_data, train_df, test_df = preprocess.preprocess(
    "TrainingSet.csv", 'TestSet.csv', limit=None, remove_low_variance=False, remove_outliers=False)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.30)

train_df['QuoteConversion_Flag'].sum()

DataFrame shape after feature selection:(78160, 32)
DataFrame shape after outlier removal:(78160, 32)


14910

## Display graphs for the lulz

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print(train_df.columns)

i = 'Geographic_info3'
plt.figure(figsize=(10,8))
plt.subplot(211)
plt.xlim(train_df[i].min(), train_df[i].max()*1.1)
ax = train_df[i].plot(kind='kde')

plt.subplot(212)
plt.xlim(train_df[i].min(), train_df[i].max()*1.1)
sns.boxplot(x=train_df[i])

## Random Forest

### Find best parameters for Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

# Random Forest
model = RandomForestClassifier(n_jobs=-1)
parameters = {'n_estimators':[1,200], 'criterion':('entropy','gini')}
gridSearch = GridSearchCV(model, parameters, cv=5)
gridSearch.fit(X_train, y_train)
#y_predict = model.predict(X_test)

gridSearch.best_params_
gridSearch.best_score_ 
gridSearch.cv_results_['params'][gridSearch.best_index_]
# accuracy_score(y_test, y_predict)

### Use Random Forest with found parameters

In [6]:
# Random Forest
randomForestModel = RandomForestClassifier(n_estimators=200, criterion='entropy',n_jobs=-1)
randomForestModel.fit(X_train, y_train)
y_predict = randomForestModel.predict(X_test)

print(accuracy_score(y_test, y_predict))

features_dict = {}
for i in range(len(randomForestModel.feature_importances_)):
    features_dict[train_df.columns[i]] = randomForestModel.feature_importances_[i]
sorted(features_dict.items(), key=lambda x:x[1], reverse=True)


0.8440378710337769


[('Sales_info1', 0.1378788210954707),
 ('Personal_info1', 0.12556944380538543),
 ('Coverage_info2', 0.04361370871523019),
 ('Field_info4', 0.03959485742627113),
 ('Sales_info2', 0.03782699869110393),
 ('Field_info2', 0.03519458822640905),
 ('Property_info4', 0.03504816370080696),
 ('Geographic_info1', 0.03477226800388598),
 ('Personal_info2', 0.033768894612445956),
 ('Sales_info4', 0.033695218323655074),
 ('Quote_ID', 0.033543543322049424),
 ('Original_Quote_Date', 0.03282808598256751),
 ('Sales_info3', 0.03199373658562435),
 ('Property_info5', 0.030497099974000737),
 ('Property_info2', 0.028996174462980414),
 ('Property_info3', 0.028386642795743138),
 ('Coverage_info1', 0.027313542368017748),
 ('Field_info1', 0.02728467399013608),
 ('Coverage_info3', 0.027039674492873325),
 ('QuoteConversion_Flag', 0.022641787811451528),
 ('NJ', 0.019923039152352096),
 ('Personal_info4', 0.0183695789439054),
 ('IL', 0.017458079795672387),
 ('CA', 0.015127171096929645),
 ('Geographic_info4', 0.01456235

In [ ]:
result_predict = randomForestModel.predict(test_data)

## k-nearest neighbour
### Find best parameters for  k-nearest neighbour

In [15]:
model = KNeighborsClassifier()
parameters = {'n_neighbors':[1,20], 'weights':('uniform', 'distance')}
gridSearch = GridSearchCV(model, parameters, cv=5)
gridSearch.fit(X_train, y_train)

gridSearch.best_params_
gridSearch.best_score_ 
gridSearch.cv_results_['params'][gridSearch.best_index_]


{'n_neighbors': 20, 'weights': 'uniform'}

### Use k-nearest neighbour with found parameters

In [4]:
KNeighbors = KNeighborsClassifier(n_neighbors=20, weights='uniform')
KNeighbors.fit(X_train, y_train)
y_predict = KNeighbors.predict(X_test)


accuracy_score(y_test, y_predict)

0.8223729102695326

In [45]:
result_predict = KNeighbors.predict(test_data)

## SVM
### Find best parameters for SVM

### Use SVM with found parameters

## Neural Network
### Find best parameters for neural network

### Use neural network with found parameters

### Save Result to file

In [3]:
import pandas as pd
test_df['QuoteConversion_Flag'] = pd.Series(result_predict, index=test_df.index)

todrop=[]
for col in test_df.columns:
    if col not in ['Quote_ID','QuoteConversion_Flag']:
        todrop.append(col)
test_df.drop(columns=todrop, inplace=True)
test_df.to_csv('Kaggle_Submission.csv', index=False)
test_df.describe()

,Quote_ID,QuoteConversion_Flag
count,26076.000000,26076.000000
mean,13038.500000,0.133916
std,7527.637146,0.340569
min,1.000000,0.000000
25%,6519.750000,0.000000
50%,13038.500000,0.000000
75%,19557.250000,0.000000
max,26076.000000,1.000000
